In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from datetime import datetime
import pandas as pd
import ast

# .env ファイルの読み込み
load_dotenv()

# APIキーの取得
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("環境変数 'OPENAI_API_KEY' が設定されていません。")

# OpenAIクライアントの初期化
client = OpenAI(api_key=api_key)

def format_debate_transcripts(content):
    """
    議論のトランスクリプトをフォーマットする関数
    
    Args:
        content: スピーチ内容のリスト
    
    Returns:
        フォーマットされたトランスクリプト文字列
    """
    if len(content) == 8:
        transcripts = f"""## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Government 3rd
{content[4]}

## Opposition 3rd
{content[5]}

## Opposition 4th
{content[6]}

## Government 4th
{content[7]}"""
    elif len(content) == 6:
        transcripts = f"""## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Opposition 3rd
{content[4]}

## Government 3rd
{content[5]}"""
    else:
        raise ValueError(f"Unexpected content length: {len(content)}")
    
    return transcripts

def create_prompt(title, content):
    """
    評価用のプロンプトを作成する関数
    
    Args:
        title: モーションのタイトル
        content: スピーチ内容のリスト
    
    Returns:
        完成したプロンプト文字列
    """
    transcripts = format_debate_transcripts(content)
    
    prompt = f"""# Instruction
Please evaluate the following parliamentary debate round based on the degree of argumentative interaction. 

## Definition of Argumentative Interaction
The goal of competitive debate is to deepen discussion through mutual understanding rather than merely defeating opponents, as debate is essentially an interactive dialogue. Thus, debaters should address opponents' arguments by presenting constructive rebuttals without ignoring their views. 

Argumentative interaction refers to how effectively debaters engage with, respond to, and build upon opposing viewpoints rather than dismissing or overlooking them. This includes:
- Direct responses to the core arguments presented by opponents
- Rebuttals that engage with the substance, logic, evidence, or assumptions of opposing arguments
- Building upon or extending arguments based on what opponents have said
- Demonstrating understanding of opponents' positions, even when disagreeing
- Creating a progressive dialogue where arguments evolve throughout the debate

Poor argumentative interaction occurs when:
- Teams talk past each other without addressing opposing arguments
- Debaters prioritize asserting their own superiority over meaningful engagement
- Key opposing arguments are ignored or mischaracterized
- The debate becomes parallel monologues rather than interactive dialogue

## Evaluation Scale
Based on your assessment of the overall debate, select one of the following ratings for the degree of argumentative interaction:

- Strongly agree
- Rather agree
- Rather disagree
- Strongly disagree

# Information
The motion for this round was: {title}

# Debate Transcripts
{transcripts}

# Output Format
Rating: [Select one: Strongly agree / Rather agree / Rather disagree / Strongly disagree]

Reasoning:
[Provide detailed explanation with specific examples from the debate showing how teams engaged with or failed to engage with each other's arguments]"""
    
    return prompt

# CSVファイルを読み込み
df = pd.read_csv('motion_title_content_english.csv')

for index, row in df.iterrows():
    id = row['id']
    title = row['Title']
    content = ast.literal_eval(row['Content'])
    
    # プロンプトの作成
    prompt = create_prompt(title, content)
    
    # OpenAI APIの呼び出し
    response = client.responses.create(
        model="o3-pro",
        input="Should we abolish nuclear power plants?",
        reasoning={
            "effort": "high"
        },
    )
    
    # 応答の表示
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]
    log_filename = f"logs/log_{id}_{title}_{current_time.replace(':', '-').replace(' ', '_').replace('.','-')}.txt"
    
    with open(log_filename, "a", encoding="utf-8") as log_file:
        log_file.write(f"Timestamp: {current_time}\n")
        log_file.write(f"Response detail: {response}\n")
        # log_file.write(f"---------------Response Content----------------\n{response.choices[0].message.content}\n")
        log_file.write(f"------------------Input Prompt---------------\n{prompt}\n-------------------\n")
    
    # 結果の表示
    print(f"ID: {id}, Title: {title}")
    print(response["output"][0]["content"][0]["text"])
    print("-" * 80)

    # 1試合のみでループを終了
    break

ID: 1, Title: The Kansai 2018 R4


TypeError: 'Response' object is not subscriptable